In [250]:
import sys
import json
import numpy as np
from scipy.optimize import minimize
import time
from itertools import combinations

# Packages for quantum stuff
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import QAOAAnsatz
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import (
    EstimatorV2 as Estimator,
    QiskitRuntimeService,
    SamplerV2 as Sampler,
)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime.fake_provider import (
    FakeBrisbane,
    FakeSherbrooke,
    FakeTorino,
)  # For simulation with realistic noise

In [251]:
# //////////    Variables    //////////
reps_p = 20
backend_simulator = AerSimulator()
#backend_simulator = AerSimulator.from_backend(FakeTorino())
instanceOfInterest = 3 #ID for the specific ising model from genereated batch
FILEDIRECTORY = "isingBatches"
isingFileName = FILEDIRECTORY + "/batch_Ising_data_TSP_9q_.json"
exactSolutionsFile = FILEDIRECTORY + "/solved_batch_Ising_data_TSP_9q_.json"

In [252]:
# //////////    Functions    //////////
def load_ising_and_build_hamiltonian(file_path, instance_id):
    """
    Loads Ising terms and weights from a JSON file.
    Determines the number of qubits from the terms and constructs
    the Hamiltonian as a Qiskit SparsePauliOp.
    """

    with open(file_path, "r") as f:
        all_isings_data = json.load(f)  # Assumes this loads a list of dicts

    selected_ising_data = None
    # Find the desired ising model within list
    for ising_instance in all_isings_data:
        if (
            ising_instance["instance_id"] == instance_id
        ):  # Assumes 'instance_id' exists and is correct
            selected_ising_data = ising_instance
            break

    terms = selected_ising_data["terms"]
    weights = selected_ising_data["weights"]
    problem_type = selected_ising_data.get("problem_type")

    pauli_list = []
    num_qubits = 0

    # Find the max number of qubits by finding the biggest index of ising variables
    all_indices = []
    for term_group in terms:
        for idx in term_group:
            all_indices.append(idx)
    num_qubits = max(all_indices) + 1

    for term_indices, weight in zip(terms, weights):
        paulis_arr = ["I"] * num_qubits
        if len(term_indices) == 1:  # Linear term
            paulis_arr[term_indices[0]] = "Z"
        elif len(term_indices) == 2:  # Quadratic term
            paulis_arr[term_indices[0]] = "Z"
            paulis_arr[term_indices[1]] = "Z"

        pauli_list.append(("".join(paulis_arr)[::-1], weight)) # how from_list works here: https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.quantum_info.SparsePauliOp
    hamiltonian = SparsePauliOp.from_list(pauli_list)
    return hamiltonian, num_qubits, problem_type

def load_file_into_dict(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

def is_valid_tsp_solution(spin_string):
    try:
        spins = [int(s) for s in spin_string.split(',')]
    except ValueError:
        return False # Handles malformed strings

    # A valid solution must have exactly 9 spins
    if len(spins) != 9:
        return False

    # Constraints on each timestep containing one city
    time_groups = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
    # Constraints on each city being visited once
    city_groups = [[0, 3, 6], [1, 4, 7], [2, 5, 8]]

    all_groups = city_groups + time_groups

    for group in all_groups:
        # Count how many '-1's are in the current group of spins
        count = sum(1 for index in group if spins[index] == -1)
        
        # If the count is not exactly 1, the constraint is violated
        if count != 1:
            return False

    return True

def calculate_ising_cost(instance_id, spin_string, all_isings_data):
    selected_instance = None
    for instance in all_isings_data:
        if instance["instance_id"] == instance_id:
            selected_instance = instance
            break

    # Parse the input spin string into a list of integers
    spin_values = [int(s) for s in spin_string.split(',')]

    # Iterate through terms and weights to calculate the energy
    total_cost = 0.0
    terms = selected_instance["terms"]
    weights = selected_instance["weights"]

    for term, weight in zip(terms, weights):
        if len(term) == 1:  # Linear term (h_i * s_i)
            i = term[0]
            total_cost += weight * spin_values[i]
        elif len(term) == 2:  # Quadratic term (J_ij * s_i * s_j)
            i, j = term[0], term[1]
            total_cost += weight * spin_values[i] * spin_values[j]

    return total_cost

def build_mixer_hamiltonian(constraints, num_qubits):
    pauli_list = []
    for group in constraints:
        # Create pairs of all qubits within the constrained group
        for qubit_pair in combinations(group, 2):
            # Create the XX term
            xx_pauli = ['I'] * num_qubits
            xx_pauli[qubit_pair[0]] = 'X'
            xx_pauli[qubit_pair[1]] = 'X'
            # Add to the list (in Qiskit's reversed order) with a coefficient of 1.0
            pauli_list.append(("".join(xx_pauli)[::-1], 1.0))

            # Create the YY term
            yy_pauli = ['I'] * num_qubits
            yy_pauli[qubit_pair[0]] = 'Y'
            yy_pauli[qubit_pair[1]] = 'Y'
            pauli_list.append(("".join(yy_pauli)[::-1], 1.0))
    mixer_hamiltonian = SparsePauliOp.from_list(pauli_list)
    return mixer_hamiltonian

def convert_basis_to_ising(solution):
    spin_list = ['+1' if bit == '0' else '-1' for bit in solution]
    
    # Join the list elements into a single, comma-separated string
    return ",".join(spin_list)

def cost_func_estimator(
    params, ansatz, estimator, cost_hamiltonian
): 
    global numOptimisations
    transpiledHamil = cost_hamiltonian.apply_layout(ansatz.layout)
    pub = (ansatz, transpiledHamil, params)

    job = estimator.run([pub])
    results = job.result()[0]
    cost = results.data.evs

    cost_float = float(np.real(cost))
    objective_func_vals.append(cost_float)

    numOptimisations = numOptimisations + 1

    return cost_float

In [253]:
costHamil, numQubits, problemType = load_ising_and_build_hamiltonian(isingFileName, instanceOfInterest)
print(costHamil)

#---- mixer experiemnet - ---
# # Each city must be visited once (rows in a 3x3 grid)
# city_constraints = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
# # Each time step can only have one city (columns in a 3x3 grid)
# time_constraints = [[0, 3, 6], [1, 4, 7], [2, 5, 8]]
# # Combine all constraint groups
# all_constraint_groups = city_constraints + time_constraints
# mixerHamil = build_mixer_hamiltonian(all_constraint_groups, numQubits)
# print(mixerHamil)

#--- inital state experiemnt---
initialCircuit = QuantumCircuit(numQubits)
initialCircuit.x([0, 4, 8])

qaoaKwargs = {
    'cost_operator': costHamil,
    'reps': reps_p
}
try:
    qaoaKwargs['mixer_operator'] = mixerHamil
except NameError:
    print("mixerHamil not defined, using default mixer.")
    
try:
    qaoaKwargs['initial_state'] = initialCircuit
except NameError:
    print("initialCircuit not defined, using default initial state.")

circuit = QAOAAnsatz(**qaoaKwargs)
circuit.measure_all()
pm = generate_preset_pass_manager(optimization_level=3, backend=backend_simulator)
candidate_circuit = pm.run(circuit)

trainedParamsAndCost = []
for i in range(1):
    num_params = 2 * reps_p
    initial_betas = (np.random.rand(reps_p) * np.pi).tolist()
    initial_gammas = (np.random.rand(reps_p) * (np.pi)).tolist()
    # initial_betas = [np.pi / 2] * reps_p
    # initial_gammas = [np.pi] * reps_p
    initial_params = initial_betas + initial_gammas #this could be an issue like if you have bad starting parameters, i would have thought over 100 problems this simple it would get it right sometime, of at least return valid tour structures
    # gammas = np.linspace(0, np.pi, reps_p)
    # betas = np.linspace(np.pi, 0, reps_p)
    # initial_params = np.concatenate([betas, gammas])    
    print(initial_params)

    objective_func_vals = []
    numOptimisations = 0
    estimator = Estimator(mode=backend_simulator)

    trainResult = minimize(
            cost_func_estimator,
            initial_params,
            args=(candidate_circuit, estimator, costHamil),
            method="COBYLA",  # Using COBYLA for gradient free optimization also fast
            tol=1e-3,
            options={"maxiter": 1000},  # Adjust as needed
        )
    print(f'{trainResult}, numLoops: {numOptimisations}') #this shows that cost and optimal parameters are super variable from run to run, so might not ever be getting a good answer because of this
    #Becomes more consitent with 10 layers, but seems like it isnt consistenly reaching global optimum, probably bc its harder to optimaise of 20 variables
    trainedParamsAndCost.append([trainResult.x, trainResult.fun])

print(trainedParamsAndCost)


SparsePauliOp(['IIIIIZIIZ', 'IIIIZIIZI', 'IIIZIIZII', 'IIZIIIIIZ', 'IZIIIIIZI', 'ZIIIIIZII', 'IIZIIZIII', 'IZIIZIIII', 'ZIIZIIIII', 'IIIIIIIZZ', 'IIIIIIZIZ', 'IIIIIIZZI', 'IIIIZZIII', 'IIIZIZIII', 'IIIZZIIII', 'IZZIIIIII', 'ZIZIIIIII', 'ZZIIIIIII', 'IIIIZIIIZ', 'IZIIIZIII', 'IIIZIIIIZ', 'ZIIIIZIII', 'IIIIIZIZI', 'IIZIZIIII', 'IIIZIIIZI', 'ZIIIZIIII', 'IIIIIZZII', 'IIZZIIIII', 'IIIIZIZII', 'IZIZIIIII', 'IIIIIIIIZ', 'IIIIIIIZI', 'IIIIIIZII', 'IIIIIZIII', 'IIIIZIIII', 'IIIZIIIII', 'IIZIIIIII', 'IZIIIIIII', 'ZIIIIIIII'],
              coeffs=[  7.5+0.j,   7.5+0.j,   7.5+0.j,   7.5+0.j,   7.5+0.j,   7.5+0.j,
   7.5+0.j,   7.5+0.j,   7.5+0.j,   7.5+0.j,   7.5+0.j,   7.5+0.j,
   7.5+0.j,   7.5+0.j,   7.5+0.j,   7.5+0.j,   7.5+0.j,   7.5+0.j,
   2. +0.j,   2. +0.j,   1.5+0.j,   1.5+0.j,   2. +0.j,   2. +0.j,
   1. +0.j,   1. +0.j,   1.5+0.j,   1.5+0.j,   1. +0.j,   1. +0.j,
 -22.5+0.j, -21.5+0.j, -22. +0.j, -22. +0.j, -21. +0.j, -20. +0.j,
 -22.5+0.j, -21.5+0.j, -22. +0.j])
[0.6271422384234363

In [254]:
bestParams = min(trainedParamsAndCost, key=lambda item: item[1])
optimized_circuit = candidate_circuit.assign_parameters(bestParams[0])
sampler = Sampler(mode=backend_simulator)
sampler.options.default_shots = 1000

sampleResult = sampler.run([optimized_circuit]).result()
dist = sampleResult[0].data.meas.get_counts()
sortedDist = sorted(dist.items(), key=lambda item: item[1], reverse=True)
print(f'Distribution: {sortedDist}')

Distribution: [('001100010', 93), ('010001100', 75), ('010100001', 64), ('100010010', 55), ('100100010', 40), ('000110001', 35), ('100011000', 32), ('011000100', 30), ('000110100', 28), ('100010100', 22), ('010010001', 21), ('110100000', 20), ('010100100', 20), ('011100000', 19), ('010101000', 19), ('001100100', 19), ('000100101', 18), ('001011000', 18), ('000100110', 15), ('100001010', 14), ('001010100', 14), ('100010001', 14), ('000101001', 14), ('010110000', 13), ('001110000', 12), ('110000001', 11), ('101000010', 11), ('000011001', 11), ('000110010', 11), ('001001001', 9), ('100100001', 9), ('000111000', 9), ('101000100', 9), ('110001000', 9), ('001000110', 8), ('011001000', 8), ('100100100', 8), ('001001100', 8), ('001010010', 7), ('001101000', 7), ('001000011', 7), ('000010011', 7), ('010100010', 7), ('100001100', 7), ('000001110', 6), ('001010001', 6), ('100001001', 6), ('010000101', 6), ('010000110', 5), ('010001001', 5), ('000001101', 5), ('001001010', 5), ('000000111', 5), ('

In [255]:
exactSolutions = load_file_into_dict(exactSolutionsFile)
mostProbableSolution = max(dist, key=dist.get)[::-1]  #Reverse the bitstring to match the big-endian convention used by dimod (q_0 leftmost bit)
#print(mostProbableSolution)

for item in exactSolutions:
    if item["instance_id"] == int(instanceOfInterest):
        correctSolutionsIsing = item["solutions"]
correctSolutionsBinary = []
for item in correctSolutionsIsing:
    correctSolutionsBinary.append(item.replace('-1', '1').replace('+1', '0').replace(',', '')) #0s and 1s may seem mixed up here, but its because the qubit state |0> corresponds to the Z-eigenvalue of +1, which is the opposite way round to the QUBO>ising mapping
    
#print(correctSolutionsBinary)

In [256]:
#want to compare the cost of solutions and also check whether quantum solution meet constraints
mostProbableIsing = convert_basis_to_ising(mostProbableSolution)
isingModels = load_file_into_dict(isingFileName)

print(f"Highest count solution (in spin variable form): {mostProbableIsing}\nGlobal Optima: {correctSolutionsIsing}")

correctSolutionCost = calculate_ising_cost(instanceOfInterest, correctSolutionsIsing[0], isingModels)
mostProbableCost = calculate_ising_cost(instanceOfInterest, mostProbableIsing, isingModels)
print(f'Cost of highest count solution: {mostProbableCost}\nGlobally minimum cost: {correctSolutionCost}')

if is_valid_tsp_solution(mostProbableIsing):
    print("The solution meets the TSP constraints.")
else:
    print("The solution does not meet the TSP constraints.")
print(f"Approximaton Ratio: {mostProbableCost / correctSolutionCost:.2f}") 

Highest count solution (in spin variable form): +1,-1,+1,+1,+1,-1,-1,+1,+1
Global Optima: ['+1,-1,+1,+1,+1,-1,-1,+1,+1', '-1,+1,+1,+1,+1,-1,+1,-1,+1']
Cost of highest count solution: -107.0
Globally minimum cost: -107.0
The solution meets the TSP constraints.
Approximaton Ratio: 1.00
